# Lab 6 - Bayesian Classifier and Gaussian Mixture Models

In the 5th assigment of the class we will study the Bayes Classifier.  

We will start with a quick demonstration of Gaussian Mixture Models with sklearn.

You have to implement two different variants of the bayes classifier using the assumption that the conditional probability P(x|y):
   1. Gaussian Bayes Classifier. P(x|y) ~ Gaussian Distribution (Normal)
   2. Gaussian Mixture Bayes Classifier. P(x|y) ~ Gaussian Mixture Model

# A. Demonstration Gaussian mixture model

#### 1 . Probability density function (pdf) of Multivariate Gaussian Distribution:

$${ p(x_{1},\ldots ,x_{k}; \mu,\Sigma)= \mathcal{N}(\mathbf {x} | \mu, \Sigma)= {\frac {\exp \left(-{\frac {1}{2}}\left({\mathbf {x} }-{\boldsymbol {\mu }}\right)^{\mathrm {T} }{\boldsymbol {\Sigma }}^{-1}\left({\mathbf {x} }-{\boldsymbol {\mu }}\right)\right)}{\sqrt {(2\pi )^{k}|{\boldsymbol {\Sigma }}|}}}}$$

$$\mathbf {x} \in \mathbb{R}^d \text{: feature vector} $$
$$\Sigma \in \mathbb{R}^{d \times d}  \text{: is the covariance matrix} $$
$$\mu \in \mathbb{R}^d \text{: mean of the distribution} $$

The value of the density at a particular point $\mathbf {x}$ represent the relative likelihood of observing that particular value in a continuous random variable's distribution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X = np.random.multivariate_normal(mean=[1, 1], 
                                  cov=[[1, 0],
                                       [0, 1]],
                                  size=200)

plt.figure()
plt.scatter(X[:, 0], X[:, 1])
plt.xlim(-4,6)
plt.ylim(-4,6)
plt.title("Data")
plt.show()
plt.close()

To infer the parameter of the distribution we can simply estimate the mean and the covarience from the data

In [ ]:
# example of the multi variate normal distribution with scipy
mean_est = np.mean(X,axis=0)
cov_est = np.cov(X, rowvar=False)

In [ ]:
mean_est

In [ ]:
cov_est

In [ ]:
from scipy.stats import multivariate_normal as mvn
dist = mvn(mean=mean_est, cov=cov_est)
x = [0, 0]
print("CDF:", dist.pdf(x))

#### 2. in the case of mixture of gaussian model the final Probability density function is as bellow:

$$p_{GMM}(x_{1},\ldots ,x_{d})= \sum_{i=1}^{k} w_k p(x_{1},\ldots ,x_{k};  \mu_i,\Sigma_i),s.t  \sum_{i=1}^{k} w_k  = 1$$


The model can be fit using the EM algorithm to find our parameters :
 1. $ w_k $ : weight of each gaussian, or the prior probability of the gaussian k, p(k)
 1. $ \mu_k $ : the mean of each the gaussian 
 1. $ \Sigma_k $: the covariance of each gaussian

lets generate some data

In [ ]:
from sklearn.datasets._samples_generator import make_blobs
import matplotlib.pyplot as plt

# generate some data
X, y_true = make_blobs(n_samples=4000, centers=4, cluster_std=0.60, random_state=0)

plt.figure()
plt.scatter(X[:, 0], X[:, 1])
plt.show()
plt.close()

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=4)
gmm.fit(X) # find the parameters of each gaussian

In [ ]:
# weights of each Gaussian i.e p(k)
w_k = gmm.weights_
w_k

In [ ]:
# mean of each gaussian
means = gmm.means_
means

In [ ]:
# mean of each gaussian
cov = gmm.covariances_
cov

In [ ]:
plt.figure()
plt.scatter(X[:, 0], X[:, 1])
plt.scatter(gmm.means_[:, 0], gmm.means_[:, 1], label="mean")
plt.legend()
plt.show()
plt.close()

# Probability Density Function of Gaussian Mixture Model

$$p_{GMM}(x_{1},\ldots ,x_{k})= \sum_{i=1}^{k} w_k p(x_{1},\ldots ,x_{k};  \mu_i,\Sigma_i),s.t  \sum_{i=1}^{k} w_k  = 1$$

In [ ]:
# pdf of gmm
x = (0, 0)


def pdf_mixture(x, w, means, cov):
    n_componets = len(means)
    p_x = 0
    for k in range(n_componets):
        p_x += w[k] * mvn(mean=means[k], cov=cov[k]).pdf(x)
    return p_x

In [ ]:
pdf_mixture(x, w_k, means, cov)

### notes 1.  if we use covariance_type="diag" the covariance metrix is diagonal

In [ ]:
gmm = GaussianMixture(n_components=4, random_state=0, covariance_type="diag")
gmm.fit(X)

In [ ]:
# so the cov in a list with the components of the diagonal
gmm.covariances_

# Assignment
## Bayesian Classifier with Gaussian Model

We begin with the implementation of the Gaussian Naive Bayes Classifier.

In the Bayesian Classifier, we use Bayes' theorem to calculate \( p(y|x) \):

$$ 
p(y|x) = \frac{p(x|y) \cdot p(y)}{p(x)} 
$$

This can be rewritten as:

$$ 
p(y|x) = \frac{p(x|y) \cdot p(y)}{\sum_{y'} p(x|y') p(y')} 
$$ 

In the Gaussian Naive Bayes Classifier, we assume that $ p(x|y) $ follows a Gaussian distribution: 

$$p(x|y) \sim \mathcal{N}(\mathbf {x} | \mu, \Sigma) $$

with the parameters $\mu$ and $\Sigma$ depending on the class $y$. 

In addition, the Naive Bayes classifier assumes **independence** of the features conditioned on the label, i.e., a **diagonal covariance matrix** for the Gaussian $ p(x|y) $.

So the Naive Bayes Classifier, you need to calculate:

1. The prior for each class $p(y)$ according to their empirical values.
2. The parameters of the Gaussian distribution for each class $y$ to model $ p(x|y) $ according to their empirical values.
3. Use Bayes' theorem to predict $ p(y|x) $.

The final prediction of our model can be the class with the maximum probability $p(y|x)$.

# Step 1.

Fill the class below and use it to:   
A. fit your model using .fit() method  
B. get the propabilities of each class for the different X using .predict_proba() method  
C. make the final predictions using the .predict() method and calculate the accuracy of your model  

Hints:
So to fit the model want:
1. Calculate the prior p(y) by calculating the percentage of samples in each class y.
2. Find the parameters (mean and covariance matrix) of the gaussian p(x|y) for **each class y**.   
We can do this by calculating the empirical mean and covariance matrix of Data X for each class y.
3. then use the bayes theorem to calculate p(y|x)

In [ ]:
class GaussianNaiveBayesClassifier:
    """
    Implementation of Naive Bayes Classifier
    """

    def __init__(self):
        # initialization of required variables.
        self.models = None  # p(x|y)
        self.prior = None  # p(y)
        self.classes = None # the classes
        self.n_classes = None # number of classes
        self.n_features = None # dimension of the features

    def fit(self, X, y):
        """
        Calculate the parameters of the gaussian models for each class y i.e P(x|y),
        as well as the prior class probabilities P(y).
        :param X: The input features
        :param y: The labels
        :return: nothing
        """
        self.n_features = X.shape[1]
        num_of_points= X.shape[0]
        
        self.classes = list(set(y))
        self.n_classes = len(set(y))
        
        #
        self.prior = np.zeros(self.n_classes)
        self.models = []
        for y_index in range(self.n_classes):
            class_y = # fill your code # set the value for y
            
            # 1. Calculate the prior P(y) for each class
            p_y = #fill your code  
            self.prior[y_index] =  p_y
            
            # 2. Estimate the parameters of the gaussian model P(x|y)
            # we will find the parameters of the gaussian for each class
            data_x_y = #fill your code        # get the data that only in class y
            
            mean_x_y = #fill your code        # calculate the mean
            cov_x_y = #fill your code         # calculate the covariance matrix, Note that you have to set rowvar=False
            diag_cov_x_y = #fill your code    # get the diagonal as the naive bayes implies
            
            gaussian = #fill your code
            self.models += [gaussian] # add the model p(x|y) in your model list
        
    def predict_proba(self, X):
        """
        Calculate the posterior probability of each class given the features, i.e. P(y|x).
        You can calculate this with Bayes's theorem:
                    P(y|x) = P(x|y) P(y) / P(x).
                    P(y|x): The Posterior probability
                    P(x|y): The Likelihood probability -  the model in self.models[y]
                    P(y): Prior probability
                    P(x): The evidence \sum_{y'} P(x|y') P(y')
        :param X:
        :return:
        """
        # get general information
        n_examples = X.shape[0]

        # fill your code to calculate P(y|x)
        posterior = 
        return posterior
    

    def predict(self, X):
        """
        Make predicted class with the maximum posterior P(y|x)
        :param X:
        :return: the predicted class
        hint: use the predict_proba class
        """
        posterior = 
        prediction = 
        return prediction

In [ ]:
# let's generate some data to use to train our models
centers = [(-10, 5), (0, 0), (0, 10), (-7, 10)]
X_y_0, _ = make_blobs(n_samples=4000, centers=centers, cluster_std=1.00, random_state=0)
centers = [(-5, 5), (-5, -2)]
X_y_1, _ = make_blobs(n_samples=2000, centers=centers, cluster_std=1.00, random_state=0)
X = np.concatenate([X_y_0, X_y_1], axis=0)
Y = np.concatenate([[0] * 4000, [1] * 2000], axis=0)

In [ ]:
# plt.scatter(X[:,0], X[:,1], color =["r" if y == 1 else "g" for y in Y], alpha = 0.5)
plt.figure()
plt.title("Scatter plot of our data")
plt.scatter(X_y_0[:, 0], X_y_0[:, 1], alpha=0.5, label="x for y=0")
plt.scatter(X_y_1[:, 0], X_y_1[:, 1], alpha=0.5, label="x for y=1")
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend()
plt.show()

In [ ]:
# initial the classifier object
gaussian_naive_bayes = GaussianNaiveBayesClassifier()

# fit the model
gaussian_naive_bayes.fit(X=X,y=Y)

In [ ]:
# get predict propabilities for each class
posterior = 
posterior

In [ ]:
# get predict propabilities for each class
y_pred = 
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true = Y, y_pred=y_pred)

In [ ]:
mean_0 = gaussian_naive_bayes.models[0].mean
X_y_pred_0 = X[y_pred==0]

mean_1 = gaussian_naive_bayes.models[1].mean
X_y_pred_1 = X[y_pred==1]

plt.figure()
plt.title("Scatter plot of our data")
plt.scatter(X_y_pred_0[:, 0], X_y_pred_0[:, 1], alpha=0.5, label="x for y_pred=0")
plt.scatter(mean_0[ 0], mean_0[ 1], label="mean for the y_pred=0", color="red")
plt.scatter(X_y_pred_1[:, 0], X_y_pred_1[:, 1], alpha=0.5, label="x for y_pred=1")
plt.scatter(mean_1[ 0], mean_1[ 1], label="mean for the y_pred=1", color="purple")
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend()
plt.show()

# Step 2.  Bayesian Classifier with Gaussian mixture model

Implement the Bayesian Classifier with Gaussian mixture model.
The model is as before, but instead of using the Gaussian Distribution to model the P(x|y) for each class, we use a mixture of Gaussian's.

So you have to find the parameters of a GMM ($w_k$, $m_k$, $cov_k$) for each different y.  
note: To fit the gmm of each class y you can use the sklearn as we demonstrate before

Fill the class below and use it to get:  
1. fit your model using .fit() method
2. the propabilities of each class for the different x using .predict_proba() method
3. make the final predictions using the .predict() method and calculate the accuracy of your model
4. compare the result with the gaussian bayes classifier

First we will create a class for GMM to have the same API as scipy multivariate gausssian in order to change the minim in our code

In [ ]:
class GMM(object):
    def __init__(self, mean, cov, w):
        self.k = len(means)  # components
        self.mean = mean
        self.cov = cov
        self.w = w

    def pdf(self, x):
        p_x = 0
        for k in range(self.k):
            p_x += self.w[k] * mvn(mean=self.mean[k], cov=self.cov[k]).pdf(x)
        return p_x

In [ ]:
class GaussianMixtureNaiveBayesClassifier:
    """
    Implementation of Gaussian Mixture Naive Bayes Classifier
    """
    def __init__(self):
        # initialization of required variables.
        self.models = None  # p(x|y)
        self.prior = None  # p(y)
        self.classes = None # the classes
        self.n_classes = None # number of classes
        self.n_features = None # dimension of the features


    def fit(self, X, y, n_components):
        """
        Calculate the parameters of the gaussian models for each class i.e P(x|y), as well as the prior class probabilities P(y).
        :param X: The input features
        :param y: The labels
        :return: nothing
        """
        self.n_features = X.shape[1]
        num_of_points= X.shape[0]
        
        self.classes = list(set(y))
        self.n_classes = len(set(y))
        
        #
        self.prior = np.zeros(self.n_classes)
        self.models = []
        for y_index in range(self.n_classes):
            
            class_y = # fill your code
            # 1. Calculate the prior P(y) for each class
            p_y = # fill your code
            self.prior[y_index] =  p_y
            
            # 2. Calculate the parameters model P(x|y)
            # we will find the parameters of the gmm for each class
            data_x_y = # fill your code        # get the data that only in class y
            gmm = # fill your code             # initialise gmm
            # fill your code                   # train gmm
            
            # get the parameters of the model
            w_y =# fill your code  
            mean_x_y =# fill your code   # calculate the mean
            diag_cov_x_y = # fill your code   # calculate the covariance matrix, Note that you have to set rowvar=False
            
            self.models += [GMM(mean=mean_x_y, cov=diag_cov_x_y, w=w_y)] # add the model p(x|y) in your model list
        
    def predict_proba(self, X):
        """
        Calculate the posterior probability of each class given the features, i.e. P(y|x).
        You can calculate this with Bayes's theorem:
                    P(y|x) = P(x|y) P(y) / P(x).
                    P(y|x): The Posterior probability
                    P(x|y): The Likelihood probability
                    P(y): Prior probability
                    P(x): The evidence \sum_{y'} P(x|y') P(y')
        :param X:
        :return:
        """
        # get general information
        n_examples = X.shape[0]

        posterior = # fill your code
        return posterior
    

    def predict(self, X):
        """
        Make predicted class with the maximum posterior P(y|x)
        :param X:
        :return: the predicted class
        hint: use the predict_proba class
        """
        posterior = # fill your code
        prediction = # fill your code
        return prediction

In [ ]:
# initial the classifier object
gaussian_naive_bayes = GaussianMixtureNaiveBayesClassifier()

# fit the model
gaussian_naive_bayes.fit(X=X,y=Y, n_components= 10)

In [ ]:
# get predict propabilities for each class
predicted_proba = 
predicted_proba

In [ ]:
# check that sum to 1, note that there may be a small discrepancies due to numerical error
predicted_proba.sum(axis=1)

In [ ]:
# get predict propabilities for each class
y_pred = 
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true = Y, y_pred=y_pred)

In [ ]:
mean_0 = gaussian_naive_bayes.models[0].mean
X_y_pred_0 = X[y_pred==0]

mean_1 = gaussian_naive_bayes.models[1].mean
X_y_pred_1 = X[y_pred==1]

plt.figure()
plt.title("Scatter plot of our data")
plt.scatter(X_y_pred_0[:, 0], X_y_pred_0[:, 1], alpha=0.5, label="x for y_pred=0")
plt.scatter(mean_0[:, 0], mean_0[:, 1], label="mean for the y_pred=0", color="red")
plt.scatter(X_y_pred_1[:, 0], X_y_pred_1[:, 1], alpha=0.5, label="x for y_pred=1")
plt.scatter(mean_1[:, 0], mean_1[:, 1], label="mean for the y_pred=1", color="purple")
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.title("Scatter plot of our data")
plt.scatter(X_y_0[:, 0], X_y_0[:, 1], alpha=0.5, label="x for y=0")
plt.scatter(X_y_1[:, 0], X_y_1[:, 1], alpha=0.5, label="x for y=1")
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend()
plt.show()